In [1]:
!git clone https://github.com/ultralytics/yolov3
!cd yolov3;pip install -qr requirements.txt

Cloning into 'yolov3'...
remote: Enumerating objects: 9862, done.
remote: Total 9862 (delta 0), reused 0 (delta 0), pack-reused 9862
Receiving objects: 100% (9862/9862), 9.19 MiB | 16.51 MiB/s, done.
Resolving deltas: 100% (6667/6667), done.
     |████████████████████████████████| 636 kB 11.9 MB/s 


In [2]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

--2021-09-27 11:12:19--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/x-gzip]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  69.7MB/s    in 11s     

2021-09-27 11:12:29 (70.4 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]

--2021-09-27 11:12:29--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19173078 (18M) [application/x-gzip]
Saving to: ‘annotations.tar.gz’

annotations.tar.gz  100%[===================>]  18.28M  59.7MB/s    in 0.3s    

2021-09-27 11:12:30 (59.7 M

In [ ]:
!mkdir /content/data
!tar -xvf images.tar.gz -C /content/data
!tar -xvf annotations.tar.gz -C /content/data

In [4]:
!mkdir /content/ox_pet;
!cd /content/ox_pet; mkdir images; mkdir labels;
!cd /content/ox_pet/images; mkdir train; mkdir val;
!cd /content/ox_pet/labels; mkdir train; mkdir val;

In [5]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

def make_train_valid_df(list_filepath, img_dir, anno_dir, test_size=0.1):
    pet_df = pd.read_csv(list_filepath, sep=' ', header=None, names=['img_name', 'class_id', 'etc1', 'etc2'])
    
    #pet_df에서 class name과 img_filepath, anno_filepath를 만든 다음에 train_test_split으로 나누고 없는거 검사해서 지운다
    pet_df['class_name'] = pet_df['img_name'].apply(lambda x:x[:x.rfind('_')])
    pet_df['img_filepath'] = img_dir + pet_df['img_name'] + '.jpg'
    pet_df['anno_filepath'] = anno_dir + pet_df['img_name'] + '.xml'
    pet_df = remove_no_annos(pet_df)

    train_df, val_df = train_test_split(pet_df, test_size=test_size, stratify=pet_df['class_id'], random_state=2021)
    return pet_df, train_df, val_df


def remove_no_annos(df):
    remove_rows = []

    for index, row in df.iterrows():
        anno_filepath = row['anno_filepath']
        if not os.path.exists(anno_filepath):
            print('index:', index, anno_filepath, '가 존재하지 않아서 삭제합니다.')
            remove_rows.append(index)
    df = df.drop(remove_rows, axis=0, inplace=False)
    return df

In [6]:
pet_df, train_df, val_df = make_train_valid_df('/content/data/annotations/trainval.txt', 
                                               '/content/data/images/', '/content/data/annotations/xmls/', test_size=0.1)

index: 4 /content/data/annotations/xmls/Abyssinian_104.xml 가 존재하지 않아서 삭제합니다.
index: 262 /content/data/annotations/xmls/Bengal_111.xml 가 존재하지 않아서 삭제합니다.
index: 1456 /content/data/annotations/xmls/samoyed_10.xml 가 존재하지 않아서 삭제합니다.
index: 2128 /content/data/annotations/xmls/Bengal_175.xml 가 존재하지 않아서 삭제합니다.
index: 2395 /content/data/annotations/xmls/Egyptian_Mau_14.xml 가 존재하지 않아서 삭제합니다.
index: 2402 /content/data/annotations/xmls/Egyptian_Mau_156.xml 가 존재하지 않아서 삭제합니다.
index: 2427 /content/data/annotations/xmls/Egyptian_Mau_186.xml 가 존재하지 않아서 삭제합니다.
index: 3177 /content/data/annotations/xmls/Ragdoll_199.xml 가 존재하지 않아서 삭제합니다.
index: 3246 /content/data/annotations/xmls/saint_bernard_15.xml 가 존재하지 않아서 삭제합니다.


In [ ]:
CLASS_NAME = pet_df['class_name'].unique().tolist()
CLASS_NAME

In [8]:
import glob
import xml.etree.ElementTree as ET

def xml_to_txt(input_xml_file, output_txt_file, object_name):
    tree = ET.parse(input_xml_file)
    root = tree.getroot()
    img_node = root.find('size')

    if img_node is None:
        return None
    
    img_width = int(img_node.find('width').text)
    img_height = int(img_node.find('height').text)

    with open(output_txt_file, 'w') as output_fpointer:
        for obj in root.findall('object'):
            xmlbox = obj.find('bndbox')
            x1 = int(xmlbox.find('xmin').text)
            y1 = int(xmlbox.find('ymin').text)
            x2 = int(xmlbox.find('xmax').text)
            y2 = int(xmlbox.find('ymax').text)
            if (x1 < 0) or (x2 < 0) or (y1 < 0) or (y2 < 0):
                break
            
            class_id, cx_norm, cy_norm, w_norm, h_norm = convert_yolo_coord(object_name, img_width, img_height, x1, y1, x2, y2)

            value_str = ('{0} {1} {2} {3} {4}').format(class_id, cx_norm, cy_norm, w_norm, h_norm)
            output_fpointer.write(value_str+'\n')

def convert_yolo_coord(object_name, img_width, img_height, x1, y1, x2, y2):
    class_id = CLASS_NAME.index(object_name)
    center_x = (x1+x2)/2
    center_y = (y1+y2)/2
    width = x2 - x1
    height =  y2 - y1

    center_x = center_x / img_width
    center_y = center_y / img_height
    width = width / img_width
    height = height / img_height

    return class_id, round(center_x, 7), round(center_y, 7), round(width, 7), round(height, 7)

In [9]:
import shutil
def make_yoloy_anno_files(df, target_image_dir, target_labels_dir):
    #df를 이용해서 한줄한줄 반복하면서 xml_to_txt함수를 실행
    for index, row in df.iterrows():
        object_name = row['class_name']
        xml_label_path = row['anno_filepath']
        src_image_path = row['img_filepath']
        target_label_path = target_labels_dir + row['img_name'] + '.txt'
        shutil.copy(src_image_path, target_image_dir)
        xml_to_txt(xml_label_path, target_label_path, object_name)

make_yoloy_anno_files(train_df, '/content/ox_pet/images/train/', '/content/ox_pet/labels/train/')
make_yoloy_anno_files(val_df, '/content/ox_pet/images/val/', '/content/ox_pet/labels/val/')

In [10]:
!wget -O /content/ox_pet/ox_pet.yaml https://raw.githubusercontent.com/chulminkw/DLCV/master/data/util/ox_pet.yaml

--2021-09-27 11:13:23--  https://raw.githubusercontent.com/chulminkw/DLCV/master/data/util/ox_pet.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 754 [text/plain]
Saving to: ‘/content/ox_pet/ox_pet.yaml’

/content/ox_pet/ox_ 100%[===================>]     754  --.-KB/s    in 0s      

2021-09-27 11:13:23 (33.1 MB/s) - ‘/content/ox_pet/ox_pet.yaml’ saved [754/754]



In [2]:
!ln -s /content/drive/My\ Drive/ /mydrive
!ls /mydrive
!mkdir "/mydrive/ultra_workdir"

ln: failed to create symbolic link '/mydrive/My Drive': File exists
 텐서플로로배우는딥러닝  'My Drive'			        ultra_workdir
'2020년 공부.zip'	  object-detection-and-tracking.ipynb   Untitled0.ipynb
'Colab Notebooks'	  pet				        공모전.zip
'Computer Vision'	  test.csv			        공부파일.zip
 MOS.zip		  tf_idf.ipynb
mkdir: cannot create directory ‘/mydrive/ultra_workdir’: File exists


In [ ]:
!cd /content/yolov3; python train.py --img 640 --batch 8 --epochs 10 --data /content/ox_pet/ox_pet.yaml --weights yolov3.pt --project /mydrive/ultra_workdir \
                                     --name pet --exist-ok 

github: up to date with https://github.com/ultralytics/yolov3 ✅
YOLOv3 🚀 v9.5.0-13-g1be3170 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=8, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='/content/ox_pet/ox_pet.yaml', device='', entity=None, epochs=10, evolve=False, exist_ok=True, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='pet', noautoanchor=False, nosave=False, notest=False, project='/mydrive/ultra_workdir', quad=False, rect=False, resume=False, save_dir='/mydrive/ultra_workdir/pet', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=8, upload_dataset=False, weights='yolov3.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir /mydrive/ultra_workdir', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.00

In [ ]:
!pip install wandb

In [5]:
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (Tesla K80)
